In [1]:
#python packages pd
import numpy as np
import matplotlib.pyplot as plt




#machine learning packages

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split



import importlib

#custom python scripts
import generator

import utilis

Using TensorFlow backend.


In [2]:
# Check that you are running GPU's
utilis.GPU_checker()





You are runnning an instance with 4 GPU's


In [3]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7fc1a02724a8>


In [4]:
%%time
#  generators
importlib.reload(generator)
training_generator = generator.Keras_DataGenerator( dataset='train', w_hyp=False)
validation_generator = generator.Keras_DataGenerator(dataset='valid', w_hyp= False)

Retrieving data from deephol-data-processed/proofs/human/train/
Generating examples from a set of 376968 examples
Retrieving data from deephol-data-processed/proofs/human/valid/
Generating examples from a set of 104054 examples
CPU times: user 2.27 s, sys: 1.41 s, total: 3.68 s
Wall time: 12.3 s


In [5]:
#Constants
VOCAB_SIZE = 1254
INPUT_LENGTH = 1000
EMBEDDING_DIM = 128

# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.2))
    model.add(CuDNNLSTM(128))
    model.add(Dense(41, activation='softmax'))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 128)         160512    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 128)         0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dense_1 (Dense)              (None, 41)                5289      
Total params: 297,897
Trainable params: 297,897
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
## WARNING IF YOU CONTAIN MULTIPLE CORE GPUS
# from keras.utils import multi_gpu_model
# model_GPU = multi_gpu_model(model, gpus= utilis.GPU_count)

In [7]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "SIMPLE_LSTM_SMALL_TANK"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [ ]:
%%time
n_epochs = 5
history = model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        verbose=1,
                        use_multiprocessing= False,
                        epochs=n_epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
5813/5814 [============================>.] - ETA: 0s - loss: 2.7165 - acc: 0.1602

In [ ]:
# save history
utilis.history_saver_bad(history, LOSS_FILE_NAME)

In [ ]:
# read numpy array
history_toplot = np.genfromtxt("training_logs/"+ LOSS_FILE_NAME +".csv")
plt.plot(history_toplot)
plt.title('Loss history')
plt.show()
